# Calibration for SSPs

In [ ]:
using CSVFiles, Query, Downloads, DataFrames, XLSX

## Socioeconomic Variables

We start by downloading the SSP Database (Shared Socioeconomic Pathways) - Version 2.0 data using the URL below to local folders _calibration/data/SSP_IAM_V2_201711_ and the country level data in _calibration/data/inputs/SspDb_country_data_2013-06-12_. Users will need to download these data on their own as it requires a (free) registration of a user account and email. At a high level, we have 5 SSPs from this source as follows.

SSP1: IMAGE (PBL)
SSP2: MESSAGE-GLOBIOM (IIASA)
SSP3: AIM (NIES)
SSP4: GCAM (PNNL)
SSP5: REMIND-MAGPIE (PIK)

Citation: Keywan Riahi, Detlef P. van Vuuren, Elmar Kriegler, Jae Edmonds, Brian C. O’Neill, Shinichiro Fujimori, Nico Bauer, Katherine Calvin, Rob Dellink, Oliver Fricko, Wolfgang Lutz, Alexander Popp, Jesus Crespo Cuaresma, Samir KC, Marian Leimbach, Leiwen Jiang, Tom Kram, Shilpa Rao, Johannes Emmerling, Kristie Ebi, Tomoko Hasegawa, Petr Havlík, Florian Humpenöder, Lara Aleluia Da Silva, Steve Smith, Elke Stehfest, Valentina Bosetti, Jiyong Eom, David Gernaat, Toshihiko Masui, Joeri Rogelj, Jessica Strefler, Laurent Drouet, Volker Krey, Gunnar Luderer, Mathijs Harmsen, Kiyoshi Takahashi, Lavinia Baumstark, Jonathan C. Doelman, Mikiko Kainuma, Zbigniew Klimont, Giacomo Marangoni, Hermann Lotze-Campen, Michael Obersteiner, Andrzej Tabeau, Massimo Tavoni. 
The Shared Socioeconomic Pathways and their energy, land use, and greenhouse gas emissions implications: An overview, Global Environmental Change, Volume 42, Pages 153-168, 2017, ISSN 0959-3780, DOI:110.1016/j.gloenvcha.2016.05.009

**SSP URL: https://tntcat.iiasa.ac.at/SspDb/dsd?Action=htmlpage&page=10**

These data are keyed by ISO 3166-1 alpha3 three-letter country codes, and we obtained a mapping key from the link below provided at the SSP URL to local folder _calibration/data/keys_.

**Country Codes Download URL: https://tntcat.iiasa.ac.at/SspDb/static/download/ISO3166-1_codes_and_country_names.xlsx**

In [ ]:
download_url = 'https://tntcat.iiasa.ac.at/SspDb/static/download/ISO3166-1_codes_and_country_names.xlsx'
path = joinpath(@__DIR__, "..", "data", "keys", "ISO3166-1_codes_and_country_names.xlsx")
Downloads.download(download_url, local_path)

The next step is to process these raw data using methodologies described [here](https://github.com/openmodels/MimiPAGE2020.jl/tree/master/preproc/growthrates) and [here](https://github.com/openmodels/SSP-Extensions) in R using the script _calibration/src/extend_ssp.R_,

Next, we postprocess the outputs of this file slightly to produce subsetted CSV files that we output to our home repository's _data`_ folder.

In [ ]:
data = load(joinpath(@__DIR__, "..", "data", "outputs", "ssp_3000_projections.csv")) |> DataFrame
for model in unique(data[!, :model]), ssp in unique(data[!, :ssp])
    data |>
        @filter(_.model == model && _.ssp == ssp && _.year <= 2500) |>
        DataFrame |>
        save(joinpath(@__DIR__, "..", "..", "data", "socioeconomic", " $(model)_$(ssp).csv"))
end

## Emissions Data

The emissions data for the runs is pulled directly from the MimiFAIRv2 repository here: https://github.com/FrankErrickson/MimiFAIRv2.jl which describes it's source as 

- Code Source: Extracted using default Python model version of FAIR2.0, available at https://github.com/njleach/FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba"
- Paper Reference: Leach et al. 2021. FaIRv2.0.0: a generalized impulse response model for climate uncertainty and future scenario exploration, Geoscientific Model Development. https://doi.org/10.5194/gmd-14-3007-2021"